# Beskrivelse av datarensing og tidshåndtering for værdata

## Introduksjon
Dette Python-skriptet bearbeider tidligere hentede værdata (nedbør, temperatur, vindhastighet, luftfuktighet) ved å korrigere tidspunktene basert på eventuelle tidsforskyvninger (`timeOffset`).  
Dataene lastes inn, prosesseres, og lagres på nytt.

---

## Funksjoner

- **Les inn data**:  
  Leser værdata fra fire separate CSV-filer.

- **Konverter tidsstempler**:  
  Konverterer kolonnen `referenceTime` fra tekst til korrekt `datetime`-format.

- **Justering av tid**:  
  Ekstraherer antall timer fra `timeOffset` og justerer `referenceTime` med denne verdien.

- **Fjerning av hjelpekollonner**:  
  Etter justeringen fjernes de midlertidige kolonnene brukt for beregning.

- **Overskriving av CSV-filer**:  
  De oppdaterte datasettene lagres tilbake i sine respektive filer.

---

## Brukte bibliotek

- `pandas` — for å lese inn, manipulere og lagre tabellbasert data
- `re` (implicit via `str.extract`) — for å trekke ut tall fra tekststrenger (regular expressions)

---

## CSV-filer som behandles

- `temperature_data.csv`
- `precipitation_data.csv`
- `wind_speed_data.csv`
- `humidity_data.csv`

---

# Oppsummering
Dette skriptet sørger for at værdataene har **riktige tidsstempler**, som tar hensyn til eventuelle **forskyvninger**.  
Dette er avgjørende for videre analyse, som f.eks. å sammenligne data på tvers av lokasjoner, modellere tidstrender eller utføre prediktiv analyse.

---

In [1]:
import pandas as pd
# Les inn data fra CSV
precipitation_data = pd.read_csv("../data/precipitation_data.csv")
temperature_data = pd.read_csv("../data/temperature_data.csv")
wind_speed_data = pd.read_csv("../data/wind_speed_data.csv")
humidity_data = pd.read_csv("../data/humidity_data.csv")

# Konverter referenceTime til datetime
precipitation_data['referenceTime'] = pd.to_datetime(precipitation_data['referenceTime'], errors='coerce')
temperature_data['referenceTime'] = pd.to_datetime(temperature_data['referenceTime'], errors='coerce')
wind_speed_data['referenceTime'] = pd.to_datetime(wind_speed_data['referenceTime'], errors='coerce')
humidity_data['referenceTime'] = pd.to_datetime(humidity_data['referenceTime'], errors='coerce')

# Henter antall timer fra timeOffset (for eksempel "PT6" -> 6)
precipitation_data['timeOffsetHours'] = precipitation_data['timeOffset'].str.extract(r'PT(\d+)').astype(float)
temperature_data['timeOffsetHours'] = temperature_data['timeOffset'].str.extract(r'PT(\d+)').astype(float)
wind_speed_data['timeOffsetHours'] = wind_speed_data['timeOffset'].str.extract(r'PT(\d+)').astype(float)
humidity_data['timeOffsetHours'] = humidity_data['timeOffset'].str.extract(r'PT(\d+)').astype(float)

# Legg til timeOffset i referenceTime
precipitation_data['referenceTime'] = precipitation_data['referenceTime'] + pd.to_timedelta(precipitation_data['timeOffsetHours'], unit='h')
temperature_data['referenceTime'] = temperature_data['referenceTime'] + pd.to_timedelta(temperature_data['timeOffsetHours'], unit='h')
wind_speed_data['referenceTime'] = wind_speed_data['referenceTime'] + pd.to_timedelta(wind_speed_data['timeOffsetHours'], unit='h')
humidity_data['referenceTime'] = humidity_data['referenceTime'] + pd.to_timedelta(humidity_data['timeOffsetHours'], unit='h')

# Slett timeOffsetHours-kolonnen
precipitation_data.drop(columns=['timeOffsetHours'], inplace=True)
temperature_data.drop(columns=['timeOffsetHours'], inplace=True)
wind_speed_data.drop(columns=['timeOffsetHours'], inplace=True)
humidity_data.drop(columns=['timeOffsetHours'], inplace=True)

# Lagre tilbake til CSV
precipitation_data.to_csv("../data/precipitation_data.csv", index=False)
temperature_data.to_csv("../data/temperature_data.csv", index=False)
wind_speed_data.to_csv("../data/wind_speed_data.csv", index=False)
humidity_data.to_csv("../data/humidity_data.csv", index=False)